<a href="https://colab.research.google.com/github/nilesh3030/Stroke-Prediction/blob/main/Notebooks/4_Weather_data_joining_with_model_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil.parser

pd.set_option ('display.max_columns', None)
pd.set_option ('display.max_rows', None)

raw_data = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_case.csv', low_memory = False)
raw_data_2021 = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/stroke_data/stroke_case_2021.csv', low_memory = False)

model_data = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/Modelling Data/model_data_2021_2022.csv', low_memory = False)

unique_patient_address = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/unique_address_with_location.csv', encoding = 'euc_kr')
unique_patient_address_2021 = pd.read_csv('/content/drive/MyDrive/Stroke_Prediction/Data/unique_address_with_location_2021.csv', encoding = 'euc_kr')
weather_data = pd.read_parquet('/content/drive/MyDrive/Stroke_Prediction/Data/Weather_Data/weather_data_2021_22_cleaned.parquet', engine = 'fastparquet')

In [5]:
unique_station = weather_data.groupby(['Branch_name']).size().reset_index(name='Freq')

unique_station.shape

(529, 2)

In [6]:
unique_patient_address_2021.shape, unique_patient_address.shape

((4505, 9), (2666, 9))

In [9]:
frames = [unique_patient_address, unique_patient_address_2021]

unique_address = pd.concat(frames)
print(unique_address.shape)
print(unique_patient_address.shape[0] + unique_patient_address_2021.shape[0])


(7171, 9)
7171


In [10]:
unique_address.head()

,Unnamed: 0,Address,Address2,Address3,Address4,Freq,Location,latitude,longitude
0,0,"강원도, 강릉시, 강동면","강원도, 강릉시",강릉시,강원도,1,"(37.7525313, 128.8759523)",37.752531,128.875952
1,1,"강원도, 강릉시, 견소동","강원도, 강릉시",강릉시,강원도,1,"(37.77614, 128.93031)",37.776140,128.930310
2,2,"강원도, 강릉시, 교동","강원도, 강릉시",강릉시,강원도,4,"(37.770061749999996, 128.8885009313695)",37.770062,128.888501
3,3,"강원도, 강릉시, 내곡동","강원도, 강릉시",강릉시,강원도,1,"(37.7384314, 128.8757664)",37.738431,128.875766
4,4,"강원도, 강릉시, 노암동","강원도, 강릉시",강릉시,강원도,6,"(37.74274, 128.91168)",37.742740,128.911680


In [11]:
# Dedup the data to handle the repeating values on branch datetime level
unique_address = unique_address.drop(['Unnamed: 0'], axis = 1)
unique_address = unique_address.drop_duplicates(subset=['Address'])

unique_address.shape

(4567, 8)

# Adding the location of weather station and assigning the weather station which is nearest from the patient's location

In [12]:
weather_station_location = pd.read_excel('/content/drive/MyDrive/Stroke_Prediction/Data/Meterological_branch_and_Location.xlsx')
weather_station_location.head()

,Branch Name,Latitude,Longitude
0,가거도,34.0728,125.1264
1,가곡,36.1250,127.6924
2,가덕,36.5461,127.5240
3,가덕도,34.9931,128.8314
4,가산,36.0956,128.5440


In [13]:
## Stripping the whitespace so that join can be done easily
weather_station_location['Branch Name'] = weather_station_location['Branch Name'].str.strip()
unique_station['Branch_name'] = unique_station['Branch_name'].str.strip()

In [14]:
## Attaching the location info to unique stations
unique_station_location = pd.merge(unique_station, weather_station_location,  how='inner', left_on=['Branch_name'], right_on = ['Branch Name'])

In [15]:
#Shape should be same as that ensures that we have location info for all the unique weather station
unique_station_location.shape

(529, 5)

In [16]:
unique_station_location = unique_station_location[['Branch Name', 'Latitude', 'Longitude']]
unique_station_location.head()

,Branch Name,Latitude,Longitude
0,가거도,34.0728,125.1264
1,가곡,36.1250,127.6924
2,가덕,36.5461,127.5240
3,가덕도,34.9931,128.8314
4,가산,36.0956,128.5440


In [17]:
### Attaching the branch which is nearest from the patient's location
import numpy as np
from scipy.spatial.distance import cdist

unique_address["Branch Name"] = pd.Series([unique_station_location["Branch Name"].iloc[np.argmin(x)] for x in cdist(unique_address[["latitude", "longitude"]], unique_station_location[["Latitude", "Longitude"]])])


In [18]:
unique_address.head()

,Address,Address2,Address3,Address4,Freq,Location,latitude,longitude,Branch Name
0,"강원도, 강릉시, 강동면","강원도, 강릉시",강릉시,강원도,1,"(37.7525313, 128.8759523)",37.752531,128.875952,강문
1,"강원도, 강릉시, 견소동","강원도, 강릉시",강릉시,강원도,1,"(37.77614, 128.93031)",37.776140,128.930310,강문
2,"강원도, 강릉시, 교동","강원도, 강릉시",강릉시,강원도,4,"(37.770061749999996, 128.8885009313695)",37.770062,128.888501,강문
3,"강원도, 강릉시, 내곡동","강원도, 강릉시",강릉시,강원도,1,"(37.7384314, 128.8757664)",37.738431,128.875766,강문
4,"강원도, 강릉시, 노암동","강원도, 강릉시",강릉시,강원도,6,"(37.74274, 128.91168)",37.742740,128.911680,강문


In [21]:
## We will create the data on 'id' level where we map the nearest branch info for each ids
id_level_data_2022 = raw_data[['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d', 'call_t']]
id_level_data_2022['Address'] = id_level_data_2022['address_si'] + ", " + id_level_data_2022['address_gu'] + ", " +  id_level_data_2022['address_dong']

id_level_data_2021 = raw_data_2021[['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d', 'call_t']]
id_level_data_2021['Address'] = id_level_data_2021['address_si'] + ", " + id_level_data_2021['address_gu'] + ", " +  id_level_data_2021['address_dong']

dfs = [id_level_data_2021, id_level_data_2022]
id_level_data = pd.concat(dfs)

print(id_level_data.shape, id_level_data_2021.shape, id_level_data_2022.shape)

<ipython-input-21-e2889129a418>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_level_data_2022['Address'] = id_level_data_2022['address_si'] + ", " + id_level_data_2022['address_gu'] + ", " +  id_level_data_2022['address_dong']


(123152, 7) (113155, 7) (9997, 7)


<ipython-input-21-e2889129a418>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_level_data_2021['Address'] = id_level_data_2021['address_si'] + ", " + id_level_data_2021['address_gu'] + ", " +  id_level_data_2021['address_dong']


In [23]:
### Dedup the data at id level
id_level_data = id_level_data.drop_duplicates(subset=['jaenan_sn'])

In [24]:
null_variables = round((id_level_data.isna().sum()/len(id_level_data))*100,2)
null_variables

jaenan_sn       0.00
address_si      8.77
address_gu      8.84
address_dong    8.77
call_d          0.01
call_t          0.01
Address         8.84
dtype: float64

In [29]:
(123152*(100-8.84))/100

112265.3632

In [25]:
nearest_branch_super = id_level_data.merge(unique_address, on = 'Address')
print(nearest_branch_super.shape)

(112212, 15)


#Processing the Nearest Branch data

In [26]:
print(nearest_branch_super.shape)
nearest_branch_super.head()

(112212, 15)


,jaenan_sn,address_si,address_gu,address_dong,call_d,call_t,Address,Address2,Address3,Address4,Freq,Location,latitude,longitude,Branch Name
0,VO1137306005,서울특별시,중구,예장동,20210123,1629,"서울특별시, 중구, 예장동","서울특별시, 중구",중구,서울특별시,1,"(37.55492, 126.98941)",37.55492,126.98941,중구
1,VW1138663575,서울특별시,중구,예장동,20210915,1838,"서울특별시, 중구, 예장동","서울특별시, 중구",중구,서울특별시,1,"(37.55492, 126.98941)",37.55492,126.98941,중구
2,VO1137314160,서울특별시,중구,예장동,20210125,1427,"서울특별시, 중구, 예장동","서울특별시, 중구",중구,서울특별시,1,"(37.55492, 126.98941)",37.55492,126.98941,중구
3,VU1138288888,서울특별시,중구,예장동,20210723,1430,"서울특별시, 중구, 예장동","서울특별시, 중구",중구,서울특별시,1,"(37.55492, 126.98941)",37.55492,126.98941,중구
4,VR1137694704,서울특별시,중구,예장동,20210419,917,"서울특별시, 중구, 예장동","서울특별시, 중구",중구,서울특별시,1,"(37.55492, 126.98941)",37.55492,126.98941,중구


In [27]:
nearest_branch_super.columns

Index(['jaenan_sn', 'address_si', 'address_gu', 'address_dong', 'call_d',
       'call_t', 'Address', 'Address2', 'Address3', 'Address4', 'Freq',
       'Location', 'latitude', 'longitude', 'Branch Name'],
      dtype='object')

In [28]:
vars = ['jaenan_sn','call_d', 'call_t', 'latitude', 'longitude', 'Branch Name']
nearest_branch = nearest_branch_super[vars]
#rename the branch name to weather station######
#'latitude', 'longitude' are the locations for the address i.e. patient location
# 'Latitude', 'Longitude' are location respective to branch address

In [29]:
convert_dict = {'call_d': str,
                'call_t': str}
 
nearest_branch = nearest_branch.astype(convert_dict)

nearest_branch["Datetime_patient"] = nearest_branch["call_d"] + nearest_branch["call_t"]

In [30]:
def convert_datetime(row):
  try:
    if len(row['Datetime_patient']) == 11 or len(row['Datetime_patient']) == 12:
      return(pd.to_datetime(row['Datetime_patient'], format = '%Y%m%d%H%M'))
  except: 
    return(np.nan)


nearest_branch['Datetime_converted'] = nearest_branch.apply(lambda row: convert_datetime(row), axis=1)

In [31]:
nearest_branch['Datetime_converted'] = nearest_branch['Datetime_converted'].dt.strftime('%Y-%m-%d %H')

In [32]:
print(nearest_branch.shape)
nearest_branch.head()

(112212, 8)


,jaenan_sn,call_d,call_t,latitude,longitude,Branch Name,Datetime_patient,Datetime_converted
0,VO1137306005,20210123,1629,37.55492,126.98941,중구,202101231629,2021-01-23 16
1,VW1138663575,20210915,1838,37.55492,126.98941,중구,202109151838,2021-09-15 18
2,VO1137314160,20210125,1427,37.55492,126.98941,중구,202101251427,2021-01-25 14
3,VU1138288888,20210723,1430,37.55492,126.98941,중구,202107231430,2021-07-23 14
4,VR1137694704,20210419,917,37.55492,126.98941,중구,20210419917,2021-04-19 09


#Combining weather and nearest-branch data

In [34]:
new_df = pd.merge(nearest_branch, weather_data,  how='left', left_on=['Branch Name','Datetime_converted'], right_on = ['Branch_name','Date_time'])
print(new_df.shape)
new_df.head()

(112212, 43)


,jaenan_sn,call_d,call_t,latitude,longitude,Branch Name,Datetime_patient,Datetime_converted,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Humidity(%),Year,Month,Date,Hour,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day,diff_Temp_max_day,diff_Temp_min_day,diff_Temp_avg_day,diff_Humidity_max_day,diff_Humidity_min_day,diff_Humidity_avg_day
0,VO1137306005,20210123,1629,37.55492,126.98941,중구,202101231629,2021-01-23 16,419.0,중구,2021-01-23 16,9.2,3.5,0.0,NaN,2021.0,1.0,2021-01-23,16.0,11.0,-19.9,-3.444953,97.8,19.0,52.504435,NaN,NaN,NaN,NaN,NaN,NaN,9.2,3.7,5.712500,NaN,NaN,NaN,3.8,0.3,1.737500,NaN,NaN,NaN
1,VW1138663575,20210915,1838,37.55492,126.98941,중구,202109151838,2021-09-15 18,419.0,중구,2021-09-15 18,23.2,4.5,0.0,51.1,2021.0,9.0,2021-09-15,18.0,29.1,15.3,20.998889,99.6,36.7,71.832500,-3.7,-1.0,-3.336864,0.4,-5.5,-3.833898,25.3,19.4,21.700000,67.3,46.0,57.758333,-0.9,0.2,-1.079167,-19.0,-0.1,-6.512500
2,VO1137314160,20210125,1427,37.55492,126.98941,중구,202101251427,2021-01-25 14,419.0,중구,2021-01-25 14,9.0,2.3,0.0,NaN,2021.0,1.0,2021-01-25,14.0,11.0,-19.9,-3.444953,97.8,19.0,52.504435,NaN,NaN,NaN,NaN,NaN,NaN,11.0,2.9,7.141667,NaN,NaN,NaN,0.1,0.8,1.058333,NaN,NaN,NaN
3,VU1138288888,20210723,1430,37.55492,126.98941,중구,202107231430,2021-07-23 14,419.0,중구,2021-07-23 14,32.9,2.1,0.0,45.2,2021.0,7.0,2021-07-23,14.0,35.5,18.5,27.035646,97.6,37.4,61.983710,3.4,6.3,5.380438,-1.8,20.2,-13.236290,33.9,25.8,29.762500,73.5,43.3,59.345833,2.3,0.0,1.422500,12.9,-4.2,4.125833
4,VR1137694704,20210419,917,37.55492,126.98941,중구,20210419917,2021-04-19 09,419.0,중구,2021-04-19 09,12.2,0.6,0.0,NaN,2021.0,4.0,2021-04-19,9.0,28.0,2.5,13.216875,98.5,14.6,53.383333,5.8,5.4,5.962305,-0.6,1.8,-6.701344,19.6,5.6,11.925000,NaN,NaN,NaN,4.6,1.2,2.950000,NaN,NaN,NaN


In [35]:
null_variables = round((new_df.isna().sum()/len(new_df))*100,2)
null_variables

jaenan_sn                   0.00
call_d                      0.00
call_t                      0.00
latitude                    0.00
longitude                   0.00
Branch Name                 0.00
Datetime_patient            0.00
Datetime_converted          4.30
Branch_code                 6.14
Branch_name                 6.14
Date_time                   6.14
Temperature                 6.32
Wind_Speed(m/s)             6.48
Precipitation(mm)           7.08
Humidity(%)                20.28
Year                        6.14
Month                       6.14
Date                        6.14
Hour                        6.14
Temp_max_month              6.14
Temp_min_month              6.14
Temp_avg_month              6.14
Humidity_max_month         14.03
Humidity_min_month         14.03
Humidity_avg_month         14.03
diff_Temp_max_month        12.84
diff_Temp_min_month        12.84
diff_Temp_avg_month        12.84
diff_Humidity_max_month    20.57
diff_Humidity_min_month    20.57
diff_Humid

In [36]:
clean_data_with_weather_variables = new_df.copy()
clean_data_with_weather_variables.shape

(112212, 43)

# Joining the model data with the cleaned weather data

In [38]:
final_data = pd.merge(model_data, clean_data_with_weather_variables,  how='inner', on = 'jaenan_sn')
print(final_data.shape)
final_data.head()

(99723, 77)


,cv_cc,cv2_cc,cv2_act,sex,stroke,cv2_cphss,triage,onset,test1,jaenan_sn,age,dbp1,sbp1,pr1,rr1,bt1,spo2_1,dbp2,sbp2,pr2,rr2,bt2,spo2_2,Weekday,Month_x,Year_x,Hour_x,Minute,Spasm,Dizziness,Faint,Consciousness,Paralysis,hy_di_disease,Disease_counts,call_d,call_t,latitude,longitude,Branch Name,Datetime_patient,Datetime_converted,Branch_code,Branch_name,Date_time,Temperature,Wind_Speed(m/s),Precipitation(mm),Humidity(%),Year_y,Month_y,Date,Hour_y,Temp_max_month,Temp_min_month,Temp_avg_month,Humidity_max_month,Humidity_min_month,Humidity_avg_month,diff_Temp_max_month,diff_Temp_min_month,diff_Temp_avg_month,diff_Humidity_max_month,diff_Humidity_min_month,diff_Humidity_avg_month,Temp_max_day,Temp_min_day,Temp_avg_day,Humidity_max_day,Humidity_min_day,Humidity_avg_day,diff_Temp_max_day,diff_Temp_min_day,diff_Temp_avg_day,diff_Humidity_max_day,diff_Humidity_min_day,diff_Humidity_avg_day
0,0,0,0,0,0,1.0,0,0,0,VO1137306005,76.0,81.0,117.0,149.0,14.0,37.1,96.0,88.0,121.0,142.0,13.0,36.9,97.0,5.0,1.0,2021.0,16.0,29.0,0,0,0,1,0,0,0,20210123,1629,37.554920,126.989410,중구,202101231629,2021-01-23 16,419.0,중구,2021-01-23 16,9.2,3.5,0.0,NaN,2021.0,1.0,2021-01-23,16.0,11.0,-19.9,-3.444953,97.8,19.0,52.504435,NaN,NaN,NaN,NaN,NaN,NaN,9.2,3.7,5.712500,NaN,NaN,NaN,3.8,0.3,1.737500,NaN,NaN,NaN
1,0,0,0,0,0,1.0,0,0,2,VP1137357129,78.0,70.0,120.0,144.0,28.0,40.0,92.0,65.0,99.0,135.0,24.0,39.5,96.0,3.0,2.0,2021.0,14.0,51.0,0,0,0,1,0,1,3,20210204,1451,37.550353,126.981563,중구,202102041451,2021-02-04 14,419.0,중구,2021-02-04 14,-2.6,2.0,0.0,NaN,2021.0,2.0,2021-02-04,14.0,16.6,-12.2,-0.290199,96.3,17.8,52.482712,5.6,7.7,3.154754,-1.5,-1.2,-0.021723,-0.3,-8.8,-4.533333,NaN,NaN,NaN,-0.8,1.2,0.454167,NaN,NaN,NaN
2,0,3,2,0,0,1.0,0,0,5,VP1137373420,50.0,118.0,227.0,101.0,15.0,37.7,98.0,121.0,213.0,92.0,15.0,37.6,99.0,0.0,2.0,2021.0,11.0,53.0,0,0,0,0,1,1,1,20210208,1153,37.671229,126.761800,김포,202102081153,2021-02-08 11,441.0,김포,2021-02-08 11,-1.9,3.9,0.0,NaN,2021.0,2.0,2021-02-08,11.0,16.2,-9.7,0.387443,NaN,NaN,NaN,3.2,7.8,3.055991,NaN,NaN,NaN,0.4,-4.9,-2.500000,NaN,NaN,NaN,-7.0,-3.0,-6.058333,NaN,NaN,NaN
3,1,2,0,1,0,0.0,1,0,7,VP1137407272,74.0,100.0,180.0,111.0,18.0,36.3,99.0,110.0,189.0,121.0,18.0,36.6,99.0,0.0,2.0,2021.0,5.0,30.0,0,0,1,0,0,1,2,20210215,530,37.550353,126.981563,중구,20210215530,2021-02-15 05,419.0,중구,2021-02-15 05,3.3,2.3,0.5,NaN,2021.0,2.0,2021-02-15,5.0,16.6,-12.2,-0.290199,96.3,17.8,52.482712,5.6,7.7,3.154754,-1.5,-1.2,-0.021723,7.0,-6.7,0.629167,NaN,NaN,NaN,-6.5,-13.9,-8.762500,NaN,NaN,NaN
4,0,3,1,1,0,1.0,0,0,8,VP1137408976,28.0,110.0,220.0,96.0,13.0,37.2,98.0,110.0,200.0,92.0,14.0,37.0,98.0,0.0,2.0,2021.0,14.0,18.0,0,0,0,0,1,1,1,20210215,1418,37.366280,127.118730,성남,202102151418,2021-02-15 14,572.0,성남,2021-02-15 14,2.6,4.2,0.0,38.9,2021.0,2.0,2021-02-15,14.0,18.6,-10.0,0.733206,90.1,9.9,48.168899,5.5,8.5,3.270798,-1.2,-2.2,-3.539104,7.5,-4.2,2.704167,82.7,31.5,53.241667,-6.8,-7.7,-5.762500,18.9,7.1,7.25


In [39]:
final_data.columns

Index(['cv_cc', 'cv2_cc', 'cv2_act', 'sex', 'stroke', 'cv2_cphss', 'triage',
       'onset', 'test1', 'jaenan_sn', 'age', 'dbp1', 'sbp1', 'pr1', 'rr1',
       'bt1', 'spo2_1', 'dbp2', 'sbp2', 'pr2', 'rr2', 'bt2', 'spo2_2',
       'Weekday', 'Month_x', 'Year_x', 'Hour_x', 'Minute', 'Spasm',
       'Dizziness', 'Faint', 'Consciousness', 'Paralysis', 'hy_di_disease',
       'Disease_counts', 'call_d', 'call_t', 'latitude', 'longitude',
       'Branch Name', 'Datetime_patient', 'Datetime_converted', 'Branch_code',
       'Branch_name', 'Date_time', 'Temperature', 'Wind_Speed(m/s)',
       'Precipitation(mm)', 'Humidity(%)', 'Year_y', 'Month_y', 'Date',
       'Hour_y', 'Temp_max_month', 'Temp_min_month', 'Temp_avg_month',
       'Humidity_max_month', 'Humidity_min_month', 'Humidity_avg_month',
       'diff_Temp_max_month', 'diff_Temp_min_month', 'diff_Temp_avg_month',
       'diff_Humidity_max_month', 'diff_Humidity_min_month',
       'diff_Humidity_avg_month', 'Temp_max_day', 'Temp_min_da

In [40]:
### There are many columns related with datetime and weather stations that would not be relevant for modelling so we will drop them

vars_to_remove = ['Weekday','Month_x','Year_x','Hour_x','Minute','call_d','call_t','latitude','longitude','Branch Name','Datetime_patient','Datetime_converted',
'Branch_code','Date_time','Year_y','Month_y','Date','Hour_y']

final_data = final_data.drop(vars_to_remove, axis = 1)
final_data.shape

(99723, 59)

In [41]:
final_data.to_csv('/content/drive/MyDrive/Stroke_Prediction/Data/Modelling Data/model_data_2021_2022_with_weather_vars.csv')